# Quick test of Excel in Python

In [1]:
%reset -f

In [2]:
from xlcalculator import ModelCompiler
from xlcalculator import Model
from xlcalculator import Evaluator

import pandas as pd

filename = r'basic.xlsx'
json_file_name = filename + r'.json'


Display the Excel file, the first row shows the formulas of the cells below.

In [36]:
pd.read_excel(filename)

,X,2X,X*X,C2,C2*2
0,1,2,1,1,2


# Turn the excel file into a python usable code

In [37]:
compiler = ModelCompiler()
new_model = compiler.read_and_parse_archive(filename, build_code=False)
new_model.persist_to_json_file(json_file_name)

In [38]:
reconstituted_model = Model()
reconstituted_model.construct_from_json_file(json_file_name, build_code=True)

evaluator = Evaluator(reconstituted_model)


In [39]:
val = evaluator.evaluate('Sheet1!B2')
print("value 'evaluated' for Sheet1!B2 before update:", val)
val = evaluator.evaluate('Sheet1!C2')
print("value 'evaluated' for Sheet1!C2 before update:", val)
val = evaluator.evaluate('Sheet1!E2')
print("value 'evaluated' for Sheet1!E2 before update:", val)

value 'evaluated' for Sheet1!B2 before update: 2
value 'evaluated' for Sheet1!C2 before update: 1
value 'evaluated' for Sheet1!E2 before update: 2


# Test: Change a cell and re evaluate the other cells

Make A2=2

In [40]:
# Now we change the value of A2
evaluator.set_cell_value('Sheet1!A2', 2)

# Evaluate the needed cells
val = evaluator.evaluate('Sheet1!B2')
print("New value for Sheet1!B2:", val)
val = evaluator.evaluate('Sheet1!C2')
print("New value for Sheet1!C2:", val)
val = evaluator.evaluate('Sheet1!E2')
print("New value for Sheet1!E2:", val)

New value for Sheet1!B2: 4
New value for Sheet1!C2: 4
New value for Sheet1!E2: 8


Make A2=3

In [41]:
# Now we change the value of A2
evaluator.set_cell_value('Sheet1!A2', 3)

# Evaluate the needed cells
val = evaluator.evaluate('Sheet1!B2')
print("New value for Sheet1!B2:", val)
val = evaluator.evaluate('Sheet1!C2')
print("New value for Sheet1!C2:", val)
val = evaluator.evaluate('Sheet1!E2')
print("New value for Sheet1!E2:", val)

New value for Sheet1!B2: 6
New value for Sheet1!C2: 9
New value for Sheet1!E2: 18


In [44]:
print(evaluator.evaluate('Sheet1!A2:C2'))

# Now try loading the Excel simulator

In [ ]:
filename = r'carlin_soskice_macroeconomic_simulator (unprotected).xlsx'
json_file_name = filename + r'.json'

compiler = ModelCompiler()
new_model = compiler.read_and_parse_archive(filename, build_code=False, ignore_hidden=True, ignore_sheets=[])
new_model.persist_to_json_file(json_file_name)

In [3]:
filename = r'carlin_soskice_macroeconomic_simulator (unprotected) (2).xlsx'
json_file_name = filename + r'.json'

compiler = ModelCompiler()


Something went wrong, but it might be fixable. **I don't know**. 


In [68]:
xlx = compiler.read_excel_file(filename)
sheetsnames = xlx.book.sheetnames


compiler.read_and_parse_archive(filename, build_code=False, ignore_sheets=sheetsnames)

ValueError: B4=1 is not a valid coordinate or range

In [67]:
sheetsnames

['main page',
 'public finance',
 'controls',
 '"normal" case',
 'exchange rates',
 'external balance',
 'gráficos PC, MR, IS',
 'inflation, income and inter-2',
 'inflation, income and interest',
 'hystere',
 'delay in perception',
 'figures on public finance',
 'numerical results',
 'input own m.p.',
 'choose m.p.',
 'choose m.p. (endogenous policy)',
 'beta estimation(inputs)',
 'loss level and beta es(outputs)',
 'loss level(inputs)',
 'compare results',
 'Hoja1']

In [66]:
from xlcalculator import patch, xltypes
from xlcalculator import reader

cells = {}
formulae = {}
ranges = {}

sheet_name = "Hoja1"
sheet = xlx.book[sheet_name]
for cell in sheet._cells.values():
    addr = f'{sheet_name}!{cell.coordinate}'

    if cell.data_type == 'f':
        formula = xltypes.XLFormula(cell.value, sheet_name)
        formulae[addr] = formula
        value = cell.cvalue
    else:
        formula = None
        value = cell.value

    cells[addr] = xltypes.XLCell(
        addr, value=value, formula=formula)


# Read Defined Names
{
    defn.name: defn.value
    for defn in xlx.book.defined_names
    if defn.hidden is None and defn.value != '#REF!'
}


ModelCompiler

<function xlcalculator.model.ModelCompiler.build_defined_names(self)>

In [6]:
from xlcalculator import patch, xltypes, evaluator, ast_nodes, parser
import logging
from xlcalculator.xlfunctions import xl, func_xltypes


In [7]:
ttt = {}
class ModelCompiler_test(ModelCompiler):     
    def read_and_parse_archive(
            self, file_name=None, ignore_sheets=[], ignore_hidden=False,
            build_code=False
    ):
        self.archive = self.read_excel_file(file_name)
        self.parse_archive(
            self.archive, ignore_sheets=ignore_sheets, ignore_hidden=ignore_hidden)

        if build_code:
            self.model.build_code()

        return self.model
    
    def parse_archive(self, archive=None, ignore_sheets=[], ignore_hidden=False):
        if archive is None:
            archive = self.archive
        else:
            self.model.cells, self.model.formulae, self.model.ranges = archive.read_cells(ignore_sheets, ignore_hidden)
            self.defined_names = archive.read_defined_names(ignore_sheets, ignore_hidden)
        self.build_defined_names()
        
        self.link_cells_to_defined_names()
        self.build_ranges()

    
    def build_defined_names(self):
        """Add defined ranges to model."""
        print("WOOOOO")
        context = evaluator.EvaluatorContext(evaluator.Evaluator(self.model), "dd!A1")
        ttt['context'] = context
        ttt['model'] = self.model
    
        for name in self.defined_names:
            print(name)
            cell_address = self.defined_names[name]
            cell_address = cell_address.replace('$', '')

            # a cell has an address like; Sheet1!A1
            if ':' not in cell_address:
                
                if cell_address not in self.model.cells:
                    logging.warning(
                        f"Defined name {name} refers to empty cell "
                        f"{cell_address}. Is not being loaded.")
                    continue

                else:
                    if self.model.cells[cell_address] is not None:
                        self.model.defined_names[name] = \
                            self.model.cells[cell_address]
            
            else:
                print(cell_address)
                ttt["defined"] = self.defined_names
                output = parser.FormulaParser() \
                    .parse(cell_address, self.defined_names, False)
                ttt["output"] = output 
                print(output )


                
                self.model.defined_names[name] = xltypes.XLRange(
                    cell_address, name=name)
                
                self.model.ranges[cell_address] = \
                    self.model.defined_names[name]
                    

            if (
                    cell_address in self.model.formulae
                    and name not in self.model.formulae
            ):
                self.model.formulae[name] = \
                    self.model.cells[cell_address].formula

test = ModelCompiler_test()
file = test.read_and_parse_archive(filename)

WOOOOO
bullets_random_shocks_gdp
IF(controls!B4=1,'"normal" case'!AX3:AX22,'"normal" case'!BB3:BB22)
IF((controls!B4) = (1), "normal" case!AX3:AX22, "normal" case!BB3:BB22)


ValueError: B4=1 is not a valid coordinate or range

In [71]:
ttt["output"]

<FunctionNode tvalue: 'IF', ttype: function, tsubtype: >

In [72]:
a = evaluator.Evaluator(model)
b = evaluator.EvaluatorContext(a, 'A1')

ttt["output"].eval(b)

<BLANK>

In [8]:
evaluatorr = ttt['context'].evaluator
context = evaluator.EvaluatorContext(evaluatorr, "main page!A1")
model = ttt['model']



In [27]:
evaluatorr.resolve_names("main page!A1")

'main page!A1'

In [9]:
c = parser.FormulaParser() \
        .parse(
                #"""IF(controls!B4=1,'"normal" case'!AX3:AX22,'"normal" case'!BB3:BB22)""",
                "'main page'!F8",
                {}, 
                False
        )
display(c)
c.eval(context)


<RangeNode tvalue: 'main page!F8', ttype: operand, tsubtype: range>

<Text 'Size'>

In [10]:
c = parser.FormulaParser() \
        .parse(
                """CONCAT('main page'!F8,2)""",
                {}, 
                False
        )
display(c)
c.eval(context)


<FunctionNode tvalue: 'CONCAT', ttype: function, tsubtype: >

<Text 'Size2'>

In [17]:
c = parser.FormulaParser() \
        .parse(
                """IF('main page'!F8="Size", 'main page'!F8:F9,2)""",
                {}, 
                False
        )
display(c)
c.eval(context)


<FunctionNode tvalue: 'IF', ttype: function, tsubtype: >

<BLANK>

In [52]:



import formulas
fpath, dir_output = 'carlin_soskice_macroeconomic_simulator (unprotected).xlsx', 'output'  
xl_model = formulas.ExcelModel().loads(fpath).finish()
xl_model.calculate()



FormulaError: ('Not a valid formula:\n%s', '=IF(controls!$B$4=1,\'"normal" case\'!$AX$3:$AX$22,\'"normal" case\'!$BB$3:$BB$22)')